<a href="https://colab.research.google.com/github/alonacode/AI/blob/main/Copy_My_of_DZ18_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оберіть одне завдання на вибір

# Завдання 1

За допомогою сегментації зображень замінітть фон для зображення https://github.com/HalyshAnton/IT-Step-Pyton-AI/blob/main/module5/images/humans.jpg

на фон з https://github.com/HalyshAnton/IT-Step-Pyton-AI/blob/main/module5/images/forest.jpg

* отримайте маску для зображення людей, де `True` - піксель класу людини, `False` - піксель іншого класу
* зробіть зображення лісу такого ж розміру як і зображення людей
* застосуйте маску щоб змінити значення пікселів зображення лісу на пікселі зображення людей
* Якщо результат поганий можете спробувати використати маску для класу `__background`

Варіант 1

In [ ]:
import torch
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from torchvision import transforms
from torchvision.io.image import read_image
import numpy as np
from PIL import Image
from torchvision.transforms import functional as F


img = read_image('/content/humans.jpg')
img_forest = read_image('/content/forest.jpg')
weights = DeepLabV3_ResNet50_Weights.DEFAULT
model = deeplabv3_resnet50(weights=weights)
model.eval()
transformer = weights.transforms(resize_size=520)
batch2 = transformer(img)
batch = batch2.unsqueeze(0)

batch2.size()

with torch.no_grad():
  pred = model(batch)['out']

# pred

In [ ]:
batch2.size()

In [ ]:
batch.size()

In [ ]:
img.size()

In [ ]:
pred.size()

In [ ]:
mask = pred.argmax(dim=1)

mask = mask[0]

In [ ]:
mask.size()

In [ ]:
mask.unique()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img.permute(1, 2, 0))

In [ ]:
plt.imshow(mask)

In [ ]:
class_to_idx = {cls: idx for (idx, cls) in enumerate(weights.meta["categories"])}
class_to_idx

In [ ]:
mask_human = mask == class_to_idx['person']
mask_human

In [ ]:
img_forest_resized = F.resize(img_forest, size=(520, 924))

img_forest_resized.size()

In [ ]:
mask_expanded = mask_human.unsqueeze(0).repeat(3, 1, 1)
mask_expanded.size()

In [ ]:
img_forest_np = np.array(img_forest_resized)
img_human_np = np.array(batch2)[:, :mask_human.shape[0], :mask_human.shape[1]]

In [ ]:
final_img = img_forest_np.copy()

In [ ]:
final_img[mask_expanded.numpy()] = img_human_np[mask_expanded.numpy()]

# 7. Виведення результату
final_img_rgb = final_img.transpose(1, 2, 0)


plt.imshow(final_img_rgb)
plt.axis('off')
plt.show()

Варіант 2

In [ ]:
import torch
import requests
import numpy as np
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.segmentation import deeplabv3_resnet101

# Завантаження зображень
img_humans = Image.open(requests.get("https://github.com/HalyshAnton/IT-Step-Pyton-AI/raw/main/module5/images/humans.jpg", stream=True).raw)
img_forest = Image.open(requests.get("https://github.com/HalyshAnton/IT-Step-Pyton-AI/raw/main/module5/images/forest.jpg", stream=True).raw)

# Перетворення до тензорів
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
img_humans_tensor = transform(img_humans).unsqueeze(0)

# Сегментаційна модель
model = deeplabv3_resnet101(pretrained=True).eval()

# Сегментація зображення
with torch.no_grad():
    output = model(img_humans_tensor)['out'][0]
output_predictions = output.argmax(0).byte().cpu().numpy()

# Отримання маски для людей
# Клас людини у COCO – 15
mask_people = (output_predictions == 15)

# Зміна розміру лісу до розмірів зображення людей
img_forest_resized = img_forest.resize(img_humans.size)
img_forest_tensor = T.ToTensor()(img_forest_resized)

# Застосування маски до зображення лісу, щоб вставити людей
result_img = img_forest_tensor.clone()
result_img[:, mask_people] = T.ToTensor()(img_humans)[:, mask_people]

# Виведення результату
result_img = result_img.permute(1, 2, 0).numpy()  # Переміщуємо осі для коректного відображення
plt.imshow(result_img)
plt.axis('off')
plt.show()


# Завдання 2

Проведіть детекцію об'єктів для відео

https://github.com/HalyshAnton/IT-Step-Pyton-AI/blob/main/module5/images/Highway%20driving%20during%20the%20day%20front%20view%20camera%20driving%20plate%204k%20footage.mp4

Підберіть параметри для якісного виявлення машин та знаків
Код для перетворення набору зображень у відео є нижче

[документація](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.ArtistAnimation.html)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab.patches import cv2_imshow
import torch
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path

# Завантаження моделі YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')  # Використовуємо модель yolov5l для кращої точності

# Налаштування параметрів моделі
model.conf = 0.25  # Поріг впевненості
model.iou = 0.5  # Поріг Intersection Over Union

# Завантаження відео для обробки
video_path = "/content/Highway driving during the day front view camera driving plate 4k footage (1).mp4"  # Шлях до вашого відео
cap = cv2.VideoCapture(video_path)

# Параметри для збереження кадрів
output_folder = "frames"
os.makedirs(output_folder, exist_ok=True)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Змінна для зберігання кадрів
output_frames = []

# Обробка кожного кадру відео
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()  # Отримуємо кадр
    if not ret:
        break  # Якщо кадр не вдалося зчитати, виходимо з циклу

    # Виконання детекції об'єктів у кадрі
    results = model(frame)

    # Малювання рамок детекції на кадрі
    results.render()
    output_frame = results.ims[0]  # Використовуємо ims замість imgs

    # Показуємо кадр після обробки
    # cv2.imshow('Processed Frame', output_frame)

    cv2_imshow(output_frame)

    # Збереження обробленого кадру як зображення
    frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, output_frame)
    output_frames.append(output_frame)  # Зберігаємо кадр у список

    # Завершення програми при натисканні клавіші 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()

print(f"Всього збережено кадрів: {frame_count}")



# Завдання 3

Згенеруйте зображення на основі тексту

[документація](https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation)

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

# pipeline = AutoPipelineForText2Image.from_pretrained(
# 	"stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16"
# ).to("cpu")
pipeline = AutoPipelineForText2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5"
).to("cpu")
generator = torch.Generator("cpu").manual_seed(31)
image = pipeline("Futuristic cityscape at dusk, glowing neon lights, flying cars, and tall skyscrapers with glass facades, soft purple and pink hues, hyper-detailed, 8k", generator=generator).images[0]
image